In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
import matplotlib.pyplot as plt

from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import transform_matrix

from utils import utils

In [ ]:
input_dir = '/run/media/hoosiki/WareHouse1/mtb/datasets/lyft-3d-od'

lyft_dataset = LyftDataset(data_path=os.path.join(input_dir, 'train'),
                           json_path=os.path.join(input_dir, 'train', 'data'),
                           verbose=True)

In [ ]:
#sample = lyft_dataset.get('sample', 'c7f7de87ec90c8993d4e7d5463208d2aa9f5ecde671960536f39b9a86f939d3c')
#sample = lyft_dataset.get('sample', '24b0962e44420e6322de3f25d9e4e5cc3c7a348ec00bfa69db21517e4ca92cc8')
sample = lyft_dataset.get('sample', '1d143ba709eaaf6a6db03a0e28d9a1665f76d6b2aa0b9816aef431a9cfbd8eca')



lidar = lyft_dataset.get('sample_data', sample['data']['LIDAR_TOP'])
lidar_data_path = lyft_dataset.get_sample_data_path(sample['data']['LIDAR_TOP'])
gt_boxes3d = lyft_dataset.get_boxes(sample['data']['LIDAR_TOP'])

ego_pose = lyft_dataset.get('ego_pose', lidar['ego_pose_token'])
calibrated_sensor = lyft_dataset.get('calibrated_sensor', lidar['calibrated_sensor_token'])

global_from_car = transform_matrix(ego_pose['translation'],
                                   Quaternion(ego_pose['rotation']),
                                   inverse=False)

car_from_sensor = transform_matrix(calibrated_sensor['translation'],
                                   Quaternion(calibrated_sensor['rotation']),
                                   inverse=False)

# pointcloud w.r.t sensor frame: [xyzi, n_points]
pointcloud = LidarPointCloud.from_file(lidar_data_path)
# pointcloud: [xyzi, n_points] -> [n_points, xyzi]
pointcloud = pointcloud.points.transpose(1, 0)

boxes = lyft_dataset.get_boxes(sample['data']['LIDAR_TOP'])

In [ ]:
pointcloud = pointcloud.transpose(1, 0)
pointcloud.shape

In [ ]:
# A sanity check, the points should be centered around 0 in car space.
plt.hist(pointcloud[0], alpha=0.5, bins=30, label="X")
plt.hist(pointcloud[1], alpha=0.5, bins=30, label="Y")
plt.hist(pointcloud[2], alpha=0.5, bins=30, label="Z")
plt.legend()
plt.xlabel("Distance from car along axis")
plt.ylabel("Amount of points")
plt.show()

In [ ]:
for i in range(3):
    mean = pointcloud[i].mean()
    std = pointcloud[i].std()
    print("mean:", mean, ", deviation:", mean-2*std, mean+2*std)

In [ ]:
gt_boxes3d = utils.convert_gt_boxes3d_from_global_to_sensor_frame(gt_boxes3d, ego_pose, calibrated_sensor)

In [ ]:
gt_boxes3d_xyzlwhr = np.array([[gt_box3d.center[0],
                                gt_box3d.center[1],
                                gt_box3d.center[2],
                                gt_box3d.wlh[1],
                                gt_box3d.wlh[0],
                                gt_box3d.wlh[2],
                                gt_box3d.orientation.yaw_pitch_roll[0]] for gt_box3d in gt_boxes3d])

In [ ]:
gt_boxes3d_xyzlwhr.shape

In [ ]:
# A sanity check, the points should be centered around 0 in car space.
plt.hist(gt_boxes3d_xyzlwhr[:, 2], alpha=0.5, bins=30, label="Z")
plt.legend()
plt.xlabel("Box3d center position in z")
plt.ylabel("Amount of points")
plt.show()

In [ ]:
gt_boxes3d_xyzlwhr[:, 2].mean()